In [ ]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx

# Load dan preprocess data
df_sales = pd.read_csv('csv/fact_sales_v2.csv', sep=';')
if df_sales['total_sales'].dtype == object:
    df_sales['total_sales'] = (
        df_sales['total_sales']
        .str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float)
)
dag_sales = df_sales.groupby('product_id', as_index=False)['total_sales'].sum()

df_order_product = pd.read_csv('csv/order_product.csv', sep=';')
df_merge1 = dag_sales.merge(
    df_order_product[['order_item_id', 'product_id']],
    on='product_id', how='left'
)

df_items = pd.read_csv('csv/order_items.csv', sep=';')
df_merge2 = df_merge1.merge(
    df_items[['order_item_id', 'order_item_name']].drop_duplicates('order_item_id'),
    on='order_item_id', how='left'
)

df_final = df_merge2[['product_id', 'order_item_name', 'total_sales']].copy()
df_final.columns = ['id', 'order_item_name', 'total_sales']
df_final = df_final.drop_duplicates(subset=['id'], keep='first')

# Ambil 5 produk teratas
df_top5 = df_final.sort_values('total_sales', ascending=False).head(5)

# Scaling ukuran bubble
sizes = (df_top5['total_sales'] / df_top5['total_sales'].max()) * 100 + 40  

# Buat graph kosong dan layout spring (bubble cloud)
G = nx.Graph()
for i in range(len(df_top5)):
    G.add_node(i)

pos = nx.spring_layout(G, k=0.5, seed=42)  # posisi node dengan spring layout

x_pos = [pos[i][0] for i in range(len(df_top5))]
y_pos = [pos[i][1] for i in range(len(df_top5))]

# Buat figure Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_pos,
    y=y_pos,
    mode='markers+text',
    text=[f"{name}<br><b>{int(total):,}</b>" for name, total in zip(df_top5['order_item_name'], df_top5['total_sales'])],
    textposition='middle center',
    marker=dict(
        size=sizes,
        color=df_top5['total_sales'],
        colorscale='Viridis',
        showscale=True,
        line=dict(width=2, color='DarkSlateGrey'),
        sizemode='diameter',
        opacity=0.7,
    ),
    hoverinfo='text',
    hovertext=[f"{name}<br>Total Sales: {total:,.0f}" for name, total in zip(df_top5['order_item_name'], df_top5['total_sales'])]
))

fig.update_layout(
    title="Bubble Cloud Chart: Top 5 Produk by Total Penjualan",
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    plot_bgcolor='white',
    height=600,
    width=800,
    margin=dict(l=20, r=20, t=60, b=20)
)

# Simpan ke PNG 
output_path = 'bubble_cloud_chart.png'
fig.write_image(output_path, scale=2)

print(f"Chart berhasil disimpan ke '{output_path}'")

fig.show()

Chart berhasil disimpan ke 'bubble_cloud_chart_top5.png'
